In [28]:
import pennylane as qml
from pennylane import numpy as pnp
import os, json
import numpy as np
from collections import Counter
import pandas as pd
from datetime import datetime

from wesszumino import build_wz_hamiltonian, pauli_str_to_op

import git
repo = git.Repo('.', search_parent_directories=True)
repo_path = repo.working_tree_dir

In [29]:
def apply_pauli_to_bitstring(pauli_str, bitstring):
    """
    Apply a Pauli string (like "IXYZ...") to a computational basis bitstring.

    Returns:
        phase (complex), out_bitstring (str)
    """
    phase = 1.0 + 0.0j
    out = list(bitstring)

    for q, (p, b_char) in enumerate(zip(pauli_str.upper(), bitstring)):
        b = 1 if b_char == "1" else 0

        if p == "I":
            continue
        elif p == "X":
            out[q] = "0" if b else "1"
        elif p == "Z":
            if b:
                phase *= -1
        elif p == "Y":
            out[q] = "0" if b else "1"
            phase *= (1j if b == 0 else -1j)  # i * (-1)^b
        else:
            raise ValueError(f"Bad Pauli char '{p}' at qubit {q}")

    return phase, "".join(out)


def reduced_matrix_from_pauli_terms(pauli_terms, basis_states):
    """
    Build reduced Hamiltonian matrix from explicit Pauli terms.

    pauli_terms: list of (coeff, pauli_str)
        e.g. [(0.5, "ZIIII"), (-1.2, "XXIYZ"), ...]
    basis_states: list of bitstrings, all same length n
        e.g. top_states from counts
    """
    basis_states = [s.strip() for s in basis_states]
    n = len(basis_states[0])
    m = len(basis_states)
    idx = {s: i for i, s in enumerate(basis_states)}

    H_red = np.zeros((m, m), dtype=complex)

    for coeff, pstr in pauli_terms:
        pstr = pstr.strip().upper()
        for ket in basis_states:
            phase, out_state = apply_pauli_to_bitstring(pstr, ket)
            if out_state in idx:
                i = idx[out_state]
                j = idx[ket]
                H_red[i, j] += coeff * phase

    return H_red


def op_to_full_pauli_string(op, wire_order):
    """
    Convert a single PennyLane Pauli word into a full string over wire_order.
    Example output: "IIXZY"
    """
    wire_pos = {w: i for i, w in enumerate(wire_order)}
    n = len(wire_order)
    chars = ["I"] * n

    def walk(o):
        # Products / tensors expose operands
        if hasattr(o, "operands") and o.operands is not None:
            for sub in o.operands:
                walk(sub)
            return

        name = o.name
        if name in ("PauliX", "X"):
            c = "X"
        elif name in ("PauliY", "Y"):
            c = "Y"
        elif name in ("PauliZ", "Z"):
            c = "Z"
        elif name in ("Identity", "I"):
            c = "I"
        else:
            raise ValueError(f"Unexpected operator in Pauli word: {name}")

        for w in o.wires:
            chars[wire_pos[w]] = c

    walk(op)
    return "".join(chars)


def pauli_terms_from_operator(H_pauli, wire_order):
    """
    Extract list of (coeff, pauli_str) from a PennyLane Hamiltonian / Sum.
    """
    if not hasattr(H_pauli, "terms"):
        raise TypeError("H_pauli has no .terms(); pass explicit pauli_terms instead.")

    coeffs, ops = H_pauli.terms()
    terms = []
    for c, o in zip(coeffs, ops):
        pstr = op_to_full_pauli_string(o, wire_order)
        terms.append((complex(c), pstr))
    return terms



In [ ]:
N = 5
a = 1.0
c = -0.8

potential = "linear"
#potential = 'quadratic'
boundary_condition = 'dirichlet'
#boundary_condition = 'periodic'

cutoff = 2
cutoffs = [2]

dt=2.0
#times = [0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0]
D=3
DS = [1,2,3,4,5,10,20]
n_steps=1
ns = [1,2,3,4,5]

all_data = []

if potential == 'quadratic':
    folder = 'C' + str(abs(c)) + '/' + 'N'+ str(N)
else:
    folder = 'N'+ str(N)

for cutoff in cutoffs:

    print(f"Cutoff: {cutoff}")

    H_path = os.path.join(repo_path, r"SUSY\Wess-Zumino\Analyses\Model Checks\HamiltonianData", boundary_condition, potential, folder, f"{potential}_{cutoff}.json")
    with open(H_path, 'r') as file:
        H_data = json.load(file)

    pauli_coeffs = H_data['pauli_coeffs']
    pauli_strings = H_data['pauli_terms']
    pauli_terms = [pauli_str_to_op(t) for t in pauli_strings]

    num_qubits = H_data['num_qubits']

    dense_H_size = H_data['H_size']

    eigenvalues = H_data['eigenvalues']
    min_eigenvalue = np.min(eigenvalues)

    print(f"Min eignvalue: {min_eigenvalue.real}")

    H_pauli = qml.Hamiltonian(pauli_coeffs, pauli_terms)


    nb = int(np.log2(cutoff))
    n = 1 + nb
    fw = [i * n for i in range(N)]

    pairs = [(fw[i], fw[i+1]) for i in range(len(fw)-1)]
    print(pairs)
  
    dev = qml.device("default.qubit", wires=num_qubits, shots=10000)
    @qml.qnode(dev)
    def circuit(t, n_steps):

        #Dirichlet-Linear
        #basis = [0]*n + [1] + [0]*nb #N2
        #basis = [0]*n + [1] + [0]*nb + [0]*n #N3
        #basis = [0]*n + [1] + [0]*nb + [0]*n + [1] + [0]*nb #N4
        basis = [0]*n + [1] + [0]*nb + [0]*n + [1] + [0]*nb + [0]*n #N5


        #Dirichlet-Linear
        print(basis)
        
        qml.BasisState(basis, wires=list(range(num_qubits)))

        for pair in pairs:
            qml.FermionicSingleExcitation(np.pi/2, wires=pair)

        qml.TrotterProduct(H_pauli, time=t, n=n_steps)

        return qml.counts(wires=list(range(num_qubits)))

    
    times = [k * dt for k in range(D)] 
    all_counts = [circuit(t=t_k, n_steps=n_steps) for t_k in times]

    samples = Counter()
    for d in all_counts:
        samples.update(d)

    sorted_states = sorted(samples.items(), key=lambda x: x[1], reverse=True)
    top_states = [s for s, c in sorted_states]
    idx = [int(s, 2) for s in top_states]

    pauli_terms = pauli_terms_from_operator(H_pauli, wire_order=list(range(num_qubits)))
    H_reduced = reduced_matrix_from_pauli_terms(pauli_terms, top_states)

    t1 = datetime.now()
    es, evs = np.linalg.eig(H_reduced)
    HRt = datetime.now() - t1

    mi = np.argmin(es)
    me = es[mi]

    row = {"potential": potential,
           "cutoff": cutoff,
           "dt": dt,
           "D": D,
           "n_steps": n_steps,
           "H_reduced_size": H_reduced.shape,
           "dense_H_size": dense_H_size,
           "H_exact_e": min_eigenvalue.real,
           "H_reduced_e": me.real,
           "diff": np.abs(min_eigenvalue-me)
           }
    
    all_data.append(row)


Cutoff: 2
Min eignvalue: -0.31590964428218754
[(0, 2), (2, 4), (4, 6), (6, 8)]
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]


In [54]:
pd.DataFrame(all_data)

,potential,cutoff,dt,D,n_steps,H_reduced_size,dense_H_size,H_exact_e,H_reduced_e,diff
0,linear,2,2.0,3,1,"(80, 80)","[1024, 1024]",-0.31591,-0.31591,2.053913e-15


In [3]:
N = 3
a = 1.0
c = -0.8

potential = "linear"
#potential = 'quadratic'
boundary_condition = 'dirichlet'
#boundary_condition = 'periodic'

cutoff = 2
cutoffs = [2]

dt=2.0
#times = [0.1, 0.2, 0.3, 0.4, 0.5, 1.0, 2.0]
D=3
DS = [1,2,3,4,5,10,20]
n_steps=1
ns = [1,2,3,4,5]

all_data = []


for cutoff in cutoffs:

    print(f"Cutoff: {cutoff}")

    H_pauli, num_qubits = build_wz_hamiltonian(
        cutoff,
        N,
        a,
        c=c,
        m=1.0,
        potential=potential,
        boundary_condition=boundary_condition,
        remove_zero_terms=True
    )

    H = qml.matrix(H_pauli, wire_order=list(range(num_qubits)))

    t1 = datetime.now()
    eigenvalues, eigenvectors = np.linalg.eig(H)
    Ht = datetime.now() - t1

    min_index = np.argmin(eigenvalues)
    min_eigenvalue = eigenvalues[min_index]

    dev = qml.device("default.qubit", wires=num_qubits, shots=10000)
    @qml.qnode(dev)
    def circuit(t, n_steps):

        #basis = [0,0,0,1,0,0,1,0,0,0,0,0]
        #basis = [0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0]
        basis = [1] + [0]*(num_qubits-1)
        qml.BasisState(basis, wires=list(range(num_qubits)))

        #qml.FermionicSingleExcitation(3.5837816671067175, wires=[0, 2])
        #qml.FermionicSingleExcitation(6.6864967676503, wires=[2, 4])

        #qml.FermionicSingleExcitation(3.5837816671067175, wires=[0, 3])
        #qml.FermionicSingleExcitation(6.6864967676503, wires=[3, 6])

        qml.FermionicSingleExcitation(3.5837816671067175, wires=[0, 4])
        qml.FermionicSingleExcitation(6.6864967676503, wires=[4, 8])

        #qml.FermionicSingleExcitation(3.5744904941650586, wires=[4, 6])
        #qml.FermionicSingleExcitation(6.689916101716141, wires=[0, 2])
        #qml.FermionicSingleExcitation(6.67626444815948, wires=[2, 4])


        qml.TrotterProduct(H_pauli, time=t, n=n_steps)

        return qml.counts(wires=list(range(num_qubits)))

    
    times = [k * dt for k in range(D)] 
    all_counts = [circuit(t=t_k, n_steps=n_steps) for t_k in times]

    samples = Counter()
    for d in all_counts:
        samples.update(d)

    sorted_states = sorted(samples.items(), key=lambda x: x[1], reverse=True)
    top_states = [s for s, c in sorted_states]
    idx = [int(s, 2) for s in top_states]

    #H_reduced = H[np.ix_(idx, idx)]
    pauli_terms = pauli_terms_from_operator(H_pauli, wire_order=list(range(num_qubits)))
    H_reduced = reduced_matrix_from_pauli_terms(pauli_terms, top_states)

    t1 = datetime.now()
    es, evs = np.linalg.eig(H_reduced)
    HRt = datetime.now() - t1

    mi = np.argmin(es)
    me = es[mi]

    row = {"potential": potential,
           "cutoff": cutoff,
           "dt": dt,
           "D": D,
           "n_steps": n_steps,
           "H_size": H.shape,
           "H_reduced_size": H_reduced.shape,
           "H_etime": str(Ht),
           "H_reduced_etime": str(HRt),
           "H_exact_e": min_eigenvalue.real,
           "H_reduced_e": me.real,
           "diff": np.abs(min_eigenvalue-me)
           }
    
    all_data.append(row)


Cutoff: 2


WireError: Cannot run circuit(s) on default.qubit as they contain wires not found on the device: {8}

In [21]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,linear,8,2.0,3,1,"(4096, 4096)","(324, 324)",0:00:22.718533,0:00:00.130867,2.735715e-08,0.00006,0.00006


: 

In [66]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,linear,4,1.0,1,1,"(4096, 4096)","(3, 3)",0:00:21.556858,0:00:00.013952,-0.000065,2.375000,2.375065
1,linear,4,1.0,2,1,"(4096, 4096)","(116, 116)",0:00:21.670361,0:00:00.037502,-0.000065,0.006632,0.006697
2,linear,4,1.0,3,1,"(4096, 4096)","(269, 269)",0:00:21.581415,0:00:00.099097,-0.000065,0.005970,0.006035


In [38]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,linear,2,1.0,3,1,"(256, 256)","(25, 25)",0:00:00.103772,0:00:00.000202,-0.207934,-0.207863,7.156063e-05
1,linear,2,1.0,3,2,"(256, 256)","(25, 25)",0:00:00.085957,0:00:00.000179,-0.207934,-0.207863,7.156063e-05
2,linear,2,1.0,3,3,"(256, 256)","(27, 27)",0:00:00.083784,0:00:00.000180,-0.207934,-0.207934,1.665335e-15
3,linear,2,1.0,3,4,"(256, 256)","(25, 25)",0:00:00.079774,0:00:00.000159,-0.207934,-0.207934,2.331468e-15
4,linear,2,1.0,3,5,"(256, 256)","(26, 26)",0:00:00.079502,0:00:00.000170,-0.207934,-0.207934,3.219647e-15


In [36]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,linear,2,1.0,1,1,"(256, 256)","(5, 5)",0:00:00.100465,0:00:00.000103,-0.207934,0.042325,2.502590e-01
1,linear,2,1.0,2,1,"(256, 256)","(18, 18)",0:00:00.090319,0:00:00.000147,-0.207934,-0.135099,7.283576e-02
2,linear,2,1.0,3,1,"(256, 256)","(25, 25)",0:00:00.082358,0:00:00.000179,-0.207934,-0.207934,1.970646e-15
3,linear,2,1.0,4,1,"(256, 256)","(27, 27)",0:00:00.080538,0:00:00.000185,-0.207934,-0.207934,2.109424e-15
4,linear,2,1.0,5,1,"(256, 256)","(29, 29)",0:00:00.080213,0:00:00.000171,-0.207934,-0.207934,2.053913e-15
5,linear,2,1.0,10,1,"(256, 256)","(30, 30)",0:00:00.081851,0:00:00.000189,-0.207934,-0.207934,1.332268e-15
6,linear,2,1.0,20,1,"(256, 256)","(33, 33)",0:00:00.079458,0:00:00.000215,-0.207934,-0.207934,2.386980e-15


In [34]:
df = pd.DataFrame(all_data)
df

,potential,cutoff,dt,D,n_steps,H_size,H_reduced_size,H_etime,H_reduced_etime,H_exact_e,H_reduced_e,diff
0,linear,2,0.1,5,1,"(256, 256)","(18, 18)",0:00:00.098169,0:00:00.000130,-0.207934,-0.129522,7.841278e-02
1,linear,2,0.2,5,1,"(256, 256)","(20, 20)",0:00:00.077444,0:00:00.000141,-0.207934,-0.171099,3.683485e-02
2,linear,2,0.3,5,1,"(256, 256)","(23, 23)",0:00:00.079689,0:00:00.000179,-0.207934,-0.184760,2.317475e-02
3,linear,2,0.4,5,1,"(256, 256)","(25, 25)",0:00:00.079773,0:00:00.000174,-0.207934,-0.207863,7.156063e-05
4,linear,2,0.5,5,1,"(256, 256)","(26, 26)",0:00:00.077793,0:00:00.000195,-0.207934,-0.207863,7.156063e-05
5,linear,2,1.0,5,1,"(256, 256)","(29, 29)",0:00:00.080802,0:00:00.000178,-0.207934,-0.207934,4.607426e-15
6,linear,2,2.0,5,1,"(256, 256)","(27, 27)",0:00:00.080279,0:00:00.000168,-0.207934,-0.207934,1.776357e-15


In [215]:
folder_path = os.path.join(repo_path, r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\SBKQD\Files\NoInitial", potential)
os.makedirs(folder_path)
df.to_excel(os.path.join(folder_path, "SBKQD.xlsx"), index=False)

In [7]:
H_pauli.ops

[I([0, 2, 4, 1, 3, 5]),
 Z(0),
 Z(2),
 Z(4),
 X(0) @ X(2),
 Y(0) @ Y(2),
 X(2) @ X(4),
 Y(2) @ Y(4),
 X(1) @ X(5)]